In [6]:
%reload_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('../'))
# from src.load_transform_data import get_new_ethereum_ohlc, get_new_ethereum_ohlc_2
from src.hopsworks_connections import pull_data, upload_data, pull_model
import pandas_ta as ta
import requests
from datetime import datetime, timedelta
import time

# Print the current working directory
current_working_directory = os.getcwd()
print("Current Working Directory:", current_working_directory)

/Users/davydsadovskyy/GitBackedShit/crypto-prediction/venv_crypto_prediction/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Current Working Directory: /Users/davydsadovskyy/GitBackedShit/crypto-prediction/notes


## Earlier, I used to scrape data from coinlore.com, but I realized they update it several hours late every day, so now I use coinGecko API instead. This was my code for that...

### Download the last 30 days of Ethereum OHLC data from coinlore.com

In [2]:
##### This is the scraping function that is used in github actions - it wont work locally unless you have chromeDriver installed #####
# But if you want ChromeDriver locally, you can do this (Mac):
# 1. brew install chromedriver
# 2. xattr -d com.apple.quarantine $(which chromedriver)
# 3. which chromedriver
# 4. the above line give the path to chromedriver. Now you can go into src/load_transfrom_data.py, and change the path in the get_new_ethereum_ohlc() function.  
# new_eth_ohlc = get_new_ethereum_ohlc()

##### Use this function when you running locally. This one doesn't require chromeDriver
# new_eth_ohlc = get_new_ethereum_ohlc_2()

# new_eth_ohlc.head()

### clean the new raw ethereum data so hopsworks accepts it

In [3]:
# def convert_value(value):
#     """
#     Converts a string value to a float. Removes $ signs, and converts
#     billion (bn), million (m), and thousand (K) values to their numeric equivalents.
#     """
#     value = value.replace('$', '')  # Remove $ sign to simplify processing
#     if value[-1].lower() == 'm':
#         return float(value[:-1]) * 1_000_000
#     elif value[-1].lower() == 'b':
#         return float(value[:-1]) * 1_000_000_000
#     elif value[-1].lower() == 'k':
#         return float(value[:-1]) * 1_000
#     elif value[-2:].lower() == 'bn':  # Handle 'bn' for billions
#         return float(value[:-2]) * 1_000_000_000
#     else:
#         return float(value)

# new_eth_ohlc.columns = [col.lower() for col in new_eth_ohlc.columns]
# new_eth_ohlc.rename(columns={'volume(eth)': 'volume_eth', 'market cap': 'market_cap'}, inplace=True)

# for col in ['open', 'high', 'low', 'close', 'volume', 'market_cap']:
#     new_eth_ohlc[col] = new_eth_ohlc[col].apply(convert_value)

# new_eth_ohlc['date'] = pd.to_datetime(new_eth_ohlc['date'])

# new_eth_ohlc.head()

### Combine this latest raw Etherum data with existing raw Ethereum data in hopsworks, and save it back into hopsworks

In [4]:
# raw_eth_ohlc = pull_data('raw_ethereum_ohlc', 1, 'raw_ethereum_ohlc_view', 1)
# raw_eth_ohlc

In [5]:
# # Hopsowrks returns fucked date column object, with weird time zone formats, we need to standardize first
# raw_eth_ohlc['date'] = pd.to_datetime(raw_eth_ohlc['date']).dt.tz_localize(None)
# new_eth_ohlc['date'] = pd.to_datetime(new_eth_ohlc['date']).dt.tz_localize(None)

# combined_raw_eth_ohlc = pd.concat([new_eth_ohlc, raw_eth_ohlc], ignore_index=True)
# combined_raw_eth_ohlc = combined_raw_eth_ohlc.drop_duplicates(subset='date', keep='last')
# combined_raw_eth_ohlc = combined_raw_eth_ohlc.sort_values(by='date')
# combined_raw_eth_ohlc

In [6]:
# upload_data(combined_raw_eth_ohlc, 'raw_ethereum_ohlc', 1, 'Raw ethereum OHLC (open, high, low, close) data from coinlore.com')

## Get recent data using CoinGecko API

In [2]:
datetime.now()

datetime.datetime(2024, 4, 22, 0, 22, 3, 257067)

In [2]:
def fetch_recent_data(crypto):
    # 20:00, or 8pm in EST is 12:00 in UTC, which is the cycle by which crypto OHLC are defined
    end_date = datetime.now().replace(hour=20, minute=0, second=0, microsecond=0)
    start_date = end_date - timedelta(days=1)

    url = f"https://api.coingecko.com/api/v3/coins/{crypto}/market_chart/range"
    params = {
        'vs_currency': 'usd',
        'from': int(start_date.timestamp()),
        'to': int(end_date.timestamp())
    }
    
    response = safe_request(url, params)
    
    if response and response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data['prices'], columns=['date', 'price'])
        df['date'] = pd.to_datetime(df['date'], unit='ms')
        df['market_cap'] = pd.DataFrame(data['market_caps'])[1].values
        df['volume_24h'] = pd.DataFrame(data['total_volumes'])[1].values
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}" if response else "Failed to fetch data; no response.")
    
    # Include only the last day
    df['date'] = pd.to_datetime(df['date'])
    # Step 2: Extract the date part from the 'datetime' column
    df['no_hour_date'] = df['date'].dt.date
    earliest_date = df['no_hour_date'].min()
    filtered_df = df[df['no_hour_date'] == earliest_date]
    filtered_df = filtered_df.drop(columns='no_hour_date')
    
    return filtered_df

def safe_request(url, params, retries=5, backoff_factor=0.5):
    for i in range(retries):
        try:
            response = requests.get(url, params=params)
            if response.status_code == 200:
                return response
            elif response.status_code == 429:
                # We're being rate-limited; back off and retry
                sleep_time = backoff_factor * (2 ** i)
                print(f"Rate limit hit. Waiting {sleep_time:.2f} seconds before retrying...")
                time.sleep(sleep_time)
            else:
                # Other errors, break the retry loop and return None
                print(f"Request failed with status code {response.status_code}.")
                return None
        except requests.exceptions.RequestException as e:
            print(f"Request exception: {e}. Retrying...")
            time.sleep(backoff_factor * (2 ** i))

    return None

In [3]:
eth_raw = fetch_recent_data("ethereum")
eth_raw

,date,price,market_cap,volume_24h
0,2024-04-22 00:00:07.304,3146.652021,3.778353e+11,9.079789e+09
1,2024-04-22 00:05:30.899,3149.335072,3.778353e+11,9.058640e+09
2,2024-04-22 00:10:45.518,3155.868424,3.789293e+11,9.195522e+09
3,2024-04-22 00:16:00.709,3162.249028,3.789293e+11,9.382324e+09
4,2024-04-22 00:20:14.557,3163.815983,3.789293e+11,8.532050e+09
...,...,...,...,...
59,2024-04-22 04:55:35.264,3196.738642,3.832988e+11,1.031924e+10
60,2024-04-22 05:00:17.254,3195.228695,3.832988e+11,1.013790e+10
61,2024-04-22 05:05:08.835,3195.956659,3.837040e+11,9.745390e+09
62,2024-04-22 05:11:10.139,3202.191747,3.837040e+11,1.018839e+10


### transform this data into ohlc (open, high, low, close) data for the past day

In [4]:
eth_new = eth_raw.resample('D', on='date').agg({
    'price': ['first', 'max', 'min', 'last', 'mean'],
    'volume_24h': 'last',
    'market_cap': 'last'
}).reset_index()
eth_new.columns = ['date', 'open', 'high', 'low', 'close', 'avg_price', 'volume', 'market_cap']
eth_new['volume_eth'] = eth_new['volume'] / eth_new['avg_price']
eth_new = eth_new[['date', 'open', 'high', 'low', 'close', 'volume', 'volume_eth', 'market_cap']]

eth_new


,date,open,high,low,close,volume,volume_eth,market_cap
0,2024-04-22,3146.652021,3207.309341,3134.798198,3203.665931,9.835554e+09,3.107154e+06,3.845108e+11


## Combine this recent data with what we already have in hopsworks feature store

In [10]:
eth_old = pull_data('ethereum_ohlc', 1, 'ethereum_ohlc_view', 2)
eth_old

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.
Feature view already existed. Skip creation.
Finished: Reading data from Hopsworks, using ArrowFlight (1.26s) 


RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/582805/featurestores/578628/featureview/ethereum_ohlc_view/version/2/trainingdatasets/version/2/statistics). Server response: 
HTTP code: 400, HTTP reason: Bad Request, body: b'Non-standard token \'NaN\': enable JsonParser.Feature.ALLOW_NON_NUMERIC_NUMBERS to allow\n at [Source: (org.glassfish.jersey.message.internal.ReaderInterceptorExecutor$UnCloseableInputStream); line: 1, column: 81626] (through reference chain: io.hops.hopsworks.common.featurestore.statistics.StatisticsDTO["featureDescriptiveStatistics"]->java.util.ArrayList[111])', error code: , error msg: , user msg: 

In [13]:
print(eth_old.columns.tolist())

NameError: name 'eth_old' is not defined

### Combine the raw, untransformed portion of the old date with the new data we just got from CoinGecko

In [8]:
columns_needed = ['date', 'open', 'high', 'low', 'close', 'volume', 'volume_eth', 'market_cap']

# Convert date columns to datetime objects, in case they are not already
eth_old['date'] = pd.to_datetime(eth_old['date'])
eth_new['date'] = pd.to_datetime(eth_new['date'])

# Remove any timezone information to make the comparison straightforward
eth_old['date'] = eth_old['date'].dt.tz_localize(None)
eth_new['date'] = eth_new['date'].dt.tz_localize(None)

dates_in_eth_old = set(eth_old['date'])

if not any(eth_new['date'].isin(dates_in_eth_old)):
    # Since eth_new's date isn't found, concatenate it
    eth_combined = pd.concat([eth_old[columns_needed], eth_new], ignore_index=True)
else:
    # If the date is found, use eth_old only
    eth_combined = eth_old[columns_needed]

eth_combined

,date,open,high,low,close,volume,volume_eth,market_cap
0,2015-11-15,0.891200,0.921500,0.875000,0.906400,4.118000e+05,4.583650e+05,6.720000e+07
1,2015-11-16,0.906200,0.944700,0.892000,0.929000,6.209000e+05,6.764420e+05,6.860000e+07
2,2015-11-17,0.924900,1.030000,0.905800,1.010000,1.100000e+06,1.183690e+06,7.220000e+07
3,2015-11-18,0.990000,1.010000,0.940500,0.990000,6.811000e+05,6.919940e+05,7.360000e+07
4,2015-11-19,0.988700,1.010000,0.937500,0.955500,4.435000e+05,4.558660e+05,7.280000e+07
...,...,...,...,...,...,...,...,...
3057,2024-04-17,3083.000000,3119.000000,2926.000000,2984.000000,1.550000e+10,5.093979e+06,3.722000e+11
3058,2024-04-18,2986.000000,3087.000000,2960.000000,3064.000000,1.350000e+10,4.453302e+06,3.706000e+11
3059,2024-04-19,3061.000000,3117.000000,2879.000000,3085.000000,1.740000e+10,5.675824e+06,3.746000e+11
3060,2024-04-20,3085.000000,3166.000000,3025.000000,3155.000000,8.900000e+09,2.884086e+06,3.776000e+11


### Use this combined raw dataset to regenerate all the response and predictor variables for the past observations, and for the new observation that was just added

In [9]:
def add_response_vars(data):

    df = data.copy()

    df["tmw_avg_high_close"] = (df["high"].shift(-1) + df["close"].shift(-1)) / 2
    df['tmw_percent_increase'] = ((df['tmw_avg_high_close'] - df['close']) / df['tmw_avg_high_close'])

    # Create binary response variable for if tomorrows return percent is positive
    df['tmw_positive_percent_increase_binary'] = (df['tmw_percent_increase'] > 0).astype(int)

    # Create binary response variables for 1-3 percent increases, by .25 steps
    intervals = [1 + i * 0.25 for i in range(int((3 - 1) / 0.25) + 1)]
    for i in intervals:
        threshold = i / 100
        column_name = f"tmw_{str(i).replace('.', '_')}_percent_increase_binary"
        df[column_name] = (df['tmw_percent_increase'] >= threshold).astype(int)

    # Last row doesn't have a value for tomorrow's return, but it was assigned 0's for response variable. Convert them to NA
    columns = [
    'tmw_positive_percent_increase_binary',
    'tmw_1_0_percent_increase_binary',
    'tmw_1_25_percent_increase_binary',
    'tmw_1_5_percent_increase_binary',
    'tmw_1_75_percent_increase_binary',
    'tmw_2_0_percent_increase_binary',
    'tmw_2_25_percent_increase_binary',
    'tmw_2_5_percent_increase_binary',
    'tmw_2_75_percent_increase_binary',
    'tmw_3_0_percent_increase_binary'
    ]

    # Set the last row's specified columns to NaN
    df.loc[df.index[-1], columns] = pd.NA

    return df

def add_predictor_vars(data):

    df = data.copy()

    ###### Predictor variables about the price momentum

    horizons = [2,5,10,25,50,100] 
    predictors = []
    for horizon in horizons:
        ### Exponential Moving Average (EMA) for past horizon
        ema_col = f"ema_{horizon}"
        df[ema_col] = df["close"] / ta.ema(df["close"], length=horizon)
        predictors.append(ema_col)

        ### Relative Strength Index (RSI) for past horizon
        rsi_col = f"rsi_{horizon}"
        df[rsi_col] = ta.rsi(df["close"], length=horizon)
        predictors.append(rsi_col)

        ### Simple Moving Average (SMA) for past horizon
        sma_col = f"sma_{horizon}"
        df[sma_col] = df["close"] / ta.sma(df["close"], length=horizon)
        predictors.append(sma_col)

        ### Sum of the number of days that had a certain percent increase for the past horizon
        columns = [
            # 'tmw_positive_percent_increase_binary',
            'tmw_1_0_percent_increase_binary',
            'tmw_1_25_percent_increase_binary',
            'tmw_1_5_percent_increase_binary',
            'tmw_1_75_percent_increase_binary',
            'tmw_2_0_percent_increase_binary',
            'tmw_2_25_percent_increase_binary',
            'tmw_2_5_percent_increase_binary',
            'tmw_2_75_percent_increase_binary',
            'tmw_3_0_percent_increase_binary'
        ]
        for col in columns:
            percent = col.split('tmw_')[1].split('_percent')[0]
            trend_col = f"last_{horizon}_day_{percent}_percent_increase_count"
            if col in df.columns:
                df[trend_col] = df[col].shift(1).rolling(horizon).sum()
            else:
                print('PREDICTORS NOT ADDED - You must run add_response_vars() first so that predictors can be created based on that info')
                break    
    
    ###### Predictor variables about seasonality/cycles

    month_names = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
    for i, month in enumerate(month_names):
        df[month] = (df['date'].dt.month == i).astype(int)

    days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
    for i, day in enumerate(days):
        df[day] = (df['date'].dt.dayofweek == i).astype(int)

    return df

In [10]:
eth_transformed = add_response_vars(eth_combined)
eth_transformed = add_predictor_vars(eth_transformed)
eth_transformed

,date,open,high,low,close,volume,volume_eth,market_cap,tmw_avg_high_close,tmw_percent_increase,...,october,november,december,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,2015-11-15,0.891200,0.921500,0.875000,0.906400,4.118000e+05,4.583650e+05,6.720000e+07,0.936850,0.032503,...,0,0,1,0,0,0,0,0,0,1
1,2015-11-16,0.906200,0.944700,0.892000,0.929000,6.209000e+05,6.764420e+05,6.860000e+07,1.020000,0.089216,...,0,0,1,1,0,0,0,0,0,0
2,2015-11-17,0.924900,1.030000,0.905800,1.010000,1.100000e+06,1.183690e+06,7.220000e+07,1.000000,-0.010000,...,0,0,1,0,1,0,0,0,0,0
3,2015-11-18,0.990000,1.010000,0.940500,0.990000,6.811000e+05,6.919940e+05,7.360000e+07,0.982750,-0.007377,...,0,0,1,0,0,1,0,0,0,0
4,2015-11-19,0.988700,1.010000,0.937500,0.955500,4.435000e+05,4.558660e+05,7.280000e+07,0.940300,-0.016165,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3057,2024-04-17,3083.000000,3119.000000,2926.000000,2984.000000,1.550000e+10,5.093979e+06,3.722000e+11,3075.500000,0.029751,...,0,0,0,0,0,1,0,0,0,0
3058,2024-04-18,2986.000000,3087.000000,2960.000000,3064.000000,1.350000e+10,4.453302e+06,3.706000e+11,3101.000000,0.011932,...,0,0,0,0,0,0,1,0,0,0
3059,2024-04-19,3061.000000,3117.000000,2879.000000,3085.000000,1.740000e+10,5.675824e+06,3.746000e+11,3160.500000,0.023889,...,0,0,0,0,0,0,0,1,0,0
3060,2024-04-20,3085.000000,3166.000000,3025.000000,3155.000000,8.900000e+09,2.884086e+06,3.776000e+11,3171.174406,0.005100,...,0,0,0,0,0,0,0,0,1,0


### Add the columns that hold the next day predictions from our model

In [12]:
# these are the columns we have to add in to eth_transformed
missing_columns = eth_old.columns.difference(eth_transformed.columns)
missing_columns

Index(['pred_tmw_1_0_percent_increase_binary',
       'pred_tmw_1_25_percent_increase_binary',
       'pred_tmw_1_5_percent_increase_binary',
       'pred_tmw_1_75_percent_increase_binary',
       'pred_tmw_2_0_percent_increase_binary',
       'pred_tmw_2_25_percent_increase_binary',
       'pred_tmw_2_5_percent_increase_binary',
       'pred_tmw_2_75_percent_increase_binary',
       'pred_tmw_3_0_percent_increase_binary',
       'pred_tmw_positive_percent_increase_binary'],
      dtype='object')

In [13]:
columns = [
    'pred_tmw_1_0_percent_increase_binary',
    'pred_tmw_1_25_percent_increase_binary',
    'pred_tmw_1_5_percent_increase_binary',
    'pred_tmw_1_75_percent_increase_binary',
    'pred_tmw_2_0_percent_increase_binary',
    'pred_tmw_2_25_percent_increase_binary',
    'pred_tmw_2_5_percent_increase_binary',
    'pred_tmw_2_75_percent_increase_binary',
    'pred_tmw_3_0_percent_increase_binary',
    'pred_tmw_positive_percent_increase_binary'
]

# Create a new DataFrame to hold these columns with an extra NA row
additional_data = eth_old[columns].copy()
additional_data.loc[len(additional_data)] = pd.NA  # Add an NA row
additional_data

eth_transformed[columns] = additional_data

eth_transformed[columns] = eth_transformed[columns].astype('Int64')

In [14]:
eth_transformed

,date,open,high,low,close,volume,volume_eth,market_cap,tmw_avg_high_close,tmw_percent_increase,...,pred_tmw_1_0_percent_increase_binary,pred_tmw_1_25_percent_increase_binary,pred_tmw_1_5_percent_increase_binary,pred_tmw_1_75_percent_increase_binary,pred_tmw_2_0_percent_increase_binary,pred_tmw_2_25_percent_increase_binary,pred_tmw_2_5_percent_increase_binary,pred_tmw_2_75_percent_increase_binary,pred_tmw_3_0_percent_increase_binary,pred_tmw_positive_percent_increase_binary
0,2015-11-15,0.891200,0.921500,0.875000,0.906400,4.118000e+05,4.583650e+05,6.720000e+07,0.936850,0.032503,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2015-11-16,0.906200,0.944700,0.892000,0.929000,6.209000e+05,6.764420e+05,6.860000e+07,1.020000,0.089216,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,2015-11-17,0.924900,1.030000,0.905800,1.010000,1.100000e+06,1.183690e+06,7.220000e+07,1.000000,-0.010000,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,2015-11-18,0.990000,1.010000,0.940500,0.990000,6.811000e+05,6.919940e+05,7.360000e+07,0.982750,-0.007377,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,2015-11-19,0.988700,1.010000,0.937500,0.955500,4.435000e+05,4.558660e+05,7.280000e+07,0.940300,-0.016165,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3057,2024-04-17,3083.000000,3119.000000,2926.000000,2984.000000,1.550000e+10,5.093979e+06,3.722000e+11,3075.500000,0.029751,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3058,2024-04-18,2986.000000,3087.000000,2960.000000,3064.000000,1.350000e+10,4.453302e+06,3.706000e+11,3101.000000,0.011932,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3059,2024-04-19,3061.000000,3117.000000,2879.000000,3085.000000,1.740000e+10,5.675824e+06,3.746000e+11,3160.500000,0.023889,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3060,2024-04-20,3085.000000,3166.000000,3025.000000,3155.000000,8.900000e+09,2.884086e+06,3.776000e+11,3171.174406,0.005100,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


## The last row has NA for response columns. Let's make the predictions for those and update those values in eth_transformed

### Get model from our Hopsworks model registry

In [15]:
MODEL_NAME = 'random_forest_ohlc' # this one predicts tmw_1_0_percent_increase_binary

# ***** later implement code that checks the latest model version and pulls that one
# def pull_model(model_name, model_version):
model = pull_model(MODEL_NAME, 1)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.


### Define predictors, the input data points, and get a prediction for tmw_1_0_percent_increase_binary

In [16]:
predictors = []
periods = [2, 5, 10, 25, 50, 100]
percent_increase_counts = ['1_0', '1_25', '1_5', '1_75', '2_0', '2_25', '2_5', '2_75', '3_0']
for period in periods:
    for indicator in ['ema', 'rsi', 'sma']:
        predictors.append(f'{indicator}_{period}')
    for percent in percent_increase_counts:
        predictors.append(f'last_{period}_day_{percent}_percent_increase_count')
months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
predictors.extend(months)
weekdays = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
predictors.extend(weekdays)

x0 = eth_transformed[predictors].iloc[-1:]
x0

,ema_2,rsi_2,sma_2,last_2_day_1_0_percent_increase_count,last_2_day_1_25_percent_increase_count,last_2_day_1_5_percent_increase_count,last_2_day_1_75_percent_increase_count,last_2_day_2_0_percent_increase_count,last_2_day_2_25_percent_increase_count,last_2_day_2_5_percent_increase_count,...,october,november,december,monday,tuesday,wednesday,thursday,friday,saturday,sunday
3061,1.002061,75.270475,0.998752,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,1


In [17]:
prediction = model.predict(x0)
prediction

array([0.])

### Add this predicted value to the last row of our eth_transformed, into the tmw_1_0_percent_increase_binary column

In [18]:
pred = int(prediction[0])
eth_transformed.at[eth_transformed.index[-1], 'pred_tmw_1_0_percent_increase_binary'] = pred

eth_transformed['pred_tmw_1_0_percent_increase_binary']

0       <NA>
1       <NA>
2       <NA>
3       <NA>
4       <NA>
        ... 
3057    <NA>
3058    <NA>
3059    <NA>
3060    <NA>
3061       0
Name: pred_tmw_1_0_percent_increase_binary, Length: 3062, dtype: Int64

In [11]:
eth_transformed

NameError: name 'eth_transformed' is not defined

## Save the data back into Hopsworks

In [21]:
columns = [
    'tmw_1_0_percent_increase_binary',
    'tmw_1_25_percent_increase_binary',
    'tmw_1_5_percent_increase_binary',
    'tmw_1_75_percent_increase_binary',
    'tmw_2_0_percent_increase_binary',
    'tmw_2_25_percent_increase_binary',
    'tmw_2_5_percent_increase_binary',
    'tmw_2_75_percent_increase_binary',
    'tmw_3_0_percent_increase_binary',
    'tmw_positive_percent_increase_binary'
]
eth_transformed[columns] = eth_transformed[columns].astype('Int64')

upload_data(eth_transformed, 'ethereum_ohlc', 1)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.


Uploading Dataframe: 100.00% |██████████| Rows 3062/3062 | Elapsed Time: 00:03 | Remaining Time: 00:00

Launching job: ethereum_ohlc_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/582805/jobs/named/ethereum_ohlc_1_offline_fg_materialization/executions
